In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import math
import h5py

import qibo
qibo.set_backend("tensorflow")
import sys
sys.path.append('../')
sys.path.append('../../')
import scripts.qkmeans as qkm
import scripts.minimization as m
import scripts.oracle as o
import scripts.grover as g
import scripts.qkmedians as qkmed
import scripts.distance_calc as distc
import utils as u
import plots as p

## Playground

### 1. Check Oracles

In [ ]:
distances = np.array([2, 7, 22, 0.5, 3.5, 15, 7])
k = len(distances)
n = int(math.floor(math.log2(k)) + 1)

In [ ]:
threshold = np.random.choice(distances)

In [ ]:
import importlib
importlib.reload(o)

In [ ]:
# threshold_oracles = o.create_oracle_threshold_set(n, distances)
# threshold_oracles

In [ ]:
#oracle_qc = o.create_lin_comb_of_oracles(n, distances, threshold, threshold_oracles)

In [ ]:
oracle_qc, indiced_to_flip_n = o.create_oracle_circ_QISKIT(distances, threshold, n)

In [ ]:
oracle_qc.to_qasm()

In [ ]:
import scripts.grover as g
qc = qibo.models.Circuit(n)
for i in range(n):
    qc.add(qibo.gates.H(i))


grover_qc = g.grover_qc(qc, n, oracle_qc, indiced_to_flip_n)
#counts = grover_qc.execute(nshots=10000).frequencies(binary=True)
        
# measure highest probability
#dictionary = counts.items()

In [ ]:
string = grover_qc.to_qasm()

In [ ]:
string

In [ ]:
string = 'qreg q[3];\ncreg register0[3];\nh q[0];\nh q[1];\nh q[2];\nmeasure q[0] -> register0[0];\nmeasure q[1] -> register0[1];\nmeasure q[2] -> register0[2];'

In [ ]:
import quimb as q
import quimb.tensor as qtn

In [ ]:
circ = qtn.Circuit(n).from_qasm(string)

## Check Dist Calc

In [ ]:
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, execute, Aer
from qiskit.tools.jupyter import *
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_textbook.tools import vector2latex
from qiskit.extensions import Initialize
from qiskit.circuit import ControlledGate

In [ ]:
import importlib
importlib.reload(distc)

In [ ]:
I = np.identity(2)
H = [[1,1], [1, -1]]
f = np.kron(H, I)
f1 = np.kron(I, I)
m = np.matrix(np.kron(f, f1))


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

min_val, max_val = -1, 1

ax.matshow(m, cmap=plt.cm.Blues)

for i in range(16):
    for j in range(16):
        c = m[j,i]
        ax.text(i, j, str(c), va='center', ha='center')

In [ ]:
a = np.array([1,5,6,1,7,4,6,9], dtype=np.float32)
b = np.array([2,3,7,4,1,2,2,2], dtype=np.float32)
num_features = 8

In [ ]:
# check dist_Calc with destructive interference
distance_TI, qc = distc.DistCalc_DI(a, b)

In [ ]:
n = int(np.log2(len(np.concatenate((a,b)))))

In [ ]:
n

In [ ]:
psi0 = qtn.MPS_computational_state("0"*n)

In [ ]:
circ = qtn.Circuit(n, psi0="0"*n, tags="PSI0")

In [ ]:
circ.apply_gate('H', 0)

In [ ]:
circ.psi

In [ ]:
circ.psi.graph(show_tags=True, show_inds=False, initial_layout="planar", iterations=100, color=["PSI0", "H"])

In [ ]:
distance = distc.DistCalc(a, b)

In [ ]:
distance

In [ ]:
n_qubits = int(math.floor(np.log2(a.shape[0]))) if float(np.log2(a.shape[0])).is_integer() \
else int(math.floor(np.log2(a.shape[0]))) + 1
n_qubits

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')
import scripts.util as ut
a_norm = ut.prepare_input(a, n_qubits, a.shape[0])
b_norm = ut.prepare_input(b, n_qubits, b.shape[0])

In [ ]:
a_norm, b_norm

In [ ]:
init_state = Initialize(a_norm)
init_state

In [ ]:
# initialize the state
init_state = Initialize(a_norm)
print(init_state.params)
print(init_state.num_qubits)
# call to generate the circuit that takes the desired vector to zero
dgc = init_state.gates_to_uncompute()

In [ ]:
#gates_to_decompose=['multiplex1_reverse_reverse', 'multiplex1_reverse']

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer
%matplotlib inline
circ = dgc.decompose()
circ2 = circ.decompose()
print(circ2.draw())
dag = circuit_to_dag(circ2)
dag_drawer(dag)

In [ ]:
circ3 = circ2.decompose()
circ4 = circ3.decompose()
circ5 = circ4.decompose()
circ5.draw()

#### What happens in gate_to_uncompute()

In [ ]:
from qiskit.visualization import plot_bloch_vector
from qiskit.circuit.library.standard_gates.x import CXGate, XGate
from qiskit.circuit.library.standard_gates.h import HGate
from qiskit.circuit.library.standard_gates.s import SGate, SdgGate
from qiskit.circuit.library.standard_gates.ry import RYGate
from qiskit.circuit.library.standard_gates.rz import RZGate

In [ ]:
def multiplex(target_gate, list_of_angles, last_cnot=True):
        """
        Return a recursive implementation of a multiplexor circuit,
        where each instruction itself has a decomposition based on
        smaller multiplexors.

        The LSB is the multiplexor "data" and the other bits are multiplexor "select".

        Args:
            target_gate (Gate): Ry or Rz gate to apply to target qubit, multiplexed
                over all other "select" qubits
            list_of_angles (list[float]): list of rotation angles to apply Ry and Rz
            last_cnot (bool): add the last cnot if last_cnot = True

        Returns:
            DAGCircuit: the circuit implementing the multiplexor's action
        """
        list_len = len(list_of_angles)
        local_num_qubits = int(math.log2(list_len)) + 1

        q = QuantumRegister(local_num_qubits)
        circuit = QuantumCircuit(q, name="multiplex" + local_num_qubits.__str__())

        lsb = q[0]
        msb = q[local_num_qubits - 1]

        # case of no multiplexing: base case for recursion
        if local_num_qubits == 1:
            circuit.append(target_gate(list_of_angles[0]), [q[0]])
            return circuit

        # calc angle weights, assuming recursion (that is the lower-level
        # requested angles have been correctly implemented by recursion
        angle_weight = np.kron([[0.5, 0.5], [0.5, -0.5]], np.identity(2 ** (local_num_qubits - 2)))
        print(f'AW: {angle_weight}')
        # calc the combo angles
        list_of_angles = angle_weight.dot(np.array(list_of_angles)).tolist()
        print(f'list_of_angles: {list_of_angles}')
        # recursive step on half the angles fulfilling the above assumption
        multiplex_1 = multiplex(target_gate, list_of_angles[0 : (list_len // 2)], False)
        circuit.append(multiplex_1.to_instruction(), q[0:-1])

        # attach CNOT as follows, thereby flipping the LSB qubit
        circuit.append(CXGate(), [msb, lsb])

        # implement extra efficiency from the paper of cancelling adjacent
        # CNOTs (by leaving out last CNOT and reversing (NOT inverting) the
        # second lower-level multiplex)
        multiplex_2 = multiplex(target_gate, list_of_angles[(list_len // 2) :], False)
        if list_len > 1:
            circuit.append(multiplex_2.to_instruction().reverse_ops(), q[0:-1])
        else:
            circuit.append(multiplex_2.to_instruction(), q[0:-1])

        # attach a final CNOT
        if last_cnot:
            circuit.append(CXGate(), [msb, lsb])

        return circuit

In [ ]:
def bloch_angles(pair_of_complex):
        """
        Static internal method to work out rotation to create the passed-in
        qubit from the zero vector.
        """
        [a_complex, b_complex] = pair_of_complex
        # Force a and b to be complex, as otherwise numpy.angle might fail.
        a_complex = complex(a_complex)
        b_complex = complex(b_complex)
        mag_a = np.absolute(a_complex)
        final_r = float(np.sqrt(mag_a ** 2 + np.absolute(b_complex) ** 2))
        if final_r < 1e-6:
            theta = 0
            phi = 0
            final_r = 0
            final_t = 0
        else:
            theta = float(2 * np.arccos(mag_a / final_r))
            a_arg = np.angle(a_complex)
            print(f'Angle of first pair: {a_arg}')
            b_arg = np.angle(b_complex)
            print(f'Angle of second pair: {b_arg}')
            final_t = a_arg + b_arg
            phi = b_arg - a_arg

        return final_r * np.exp(1.0j * final_t / 2), theta, phi

In [ ]:
from qiskit.visualization import plot_bloch_vector

def rotations_to_disentangle(local_param):
    """
    Static internal method to work out Ry and Rz rotation angles used
    to disentangle the LSB qubit.
    These rotations make up the block diagonal matrix U (i.e. multiplexor)
    that disentangles the LSB.

    [[Ry(theta_1).Rz(phi_1)  0   .   .   0],
     [0         Ry(theta_2).Rz(phi_2) .  0],
                                .
                                    .
      0         0           Ry(theta_2^n).Rz(phi_2^n)]]
    """
    remaining_vector = []
    thetas = []
    phis = []

    param_len = len(local_param)

    for i in range(param_len // 2):
        # Ry and Rz rotations to move bloch vector from 0 to "imaginary"
        # qubit
        # (imagine a qubit state signified by the amplitudes at index 2*i
        # and 2*(i+1), corresponding to the select qubits of the
        # multiplexor being in state |i>)
        print('=======================================')
        print(f'Using these inputs: {local_param[2 * i : 2 * (i + 1)]}')
        (remains, add_theta, add_phi) = bloch_angles(
            local_param[2 * i : 2 * (i + 1)]
        )
        print(f'Set bloch angles for inputs: {remains}, theta={-add_theta}, phi={-add_phi}')
        remaining_vector.append(remains)

        # rotations for all imaginary qubits of the full vector
        # to move from where it is to zero, hence the negative sign
        thetas.append(-add_theta)
        phis.append(-add_phi)
    #plot_bloch_vector(remaining_vector)
    return remaining_vector, thetas, phis

In [ ]:
q = QuantumRegister(n_qubits)
circuit = QuantumCircuit(q, name="disentangler")

# kick start the peeling loop, and disentangle one-by-one from LSB to MSB
remaining_param = init_state.params

for i in range(n_qubits):
    print(f'------------FOR QUBIT NUM. {i} ----------')
    # work out which rotations must be done to disentangle the LSB
    # qubit (we peel away one qubit at a time)
    (remaining_param, thetas, phis) = rotations_to_disentangle(remaining_param)
    print(remaining_param)
    print(thetas)
    print(phis)
    # perform the required rotations to decouple the LSB qubit (so that
    # it can be "factored" out, leaving a shorter amplitude vector to peel away)
    add_last_cnot = True
    if np.linalg.norm(phis) != 0 and np.linalg.norm(thetas) != 0:
        add_last_cnot = False

    if np.linalg.norm(phis) != 0:
        rz_mult = multiplex(RZGate, phis, last_cnot=add_last_cnot)
        print(f'Apply RZ gate')
        print(rz_mult.decompose())
        circuit.append(rz_mult.to_instruction(), q[i : n_qubits])

    if np.linalg.norm(thetas) != 0:
        ry_mult = multiplex(RYGate, thetas, last_cnot=add_last_cnot)
        print(f'Apply RY gate')
        print(ry_mult.decompose())
        circuit.append(ry_mult.to_instruction().reverse_ops(), q[i : n_qubits])
circuit.global_phase -= np.angle(sum(remaining_param))
print('TOTAL CIRCUIT')
print(circuit.decompose().decompose().decompose().decompose().decompose().draw())

In [ ]:
initialize_instr = circ2.to_instruction().inverse()
initialize_instr

In [ ]:
from qiskit.quantum_info import Statevector
# q = QuantumRegister(n_qubits, "q")
# initialize_circuit = QuantumCircuit(q)
# initialize_circuit.initialize(a_norm, q[:])
# initialize_inst = initialize_circuit.to_instruction()

#initialize_circuit.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})
q1 = QuantumRegister(n_qubits, "q")
circuit_to_gate = QuantumCircuit(q1, name="x_init")
circuit_to_gate.append(initialize_instr, q1[:])
state_vec = Statevector.from_instruction(circuit_to_gate).data
print(state_vec)
gate = circuit_to_gate.to_gate()
print(circuit_to_gate.decompose().decompose().decompose().decompose())

In [ ]:
from qiskit.visualization import plot_state_qsphere
%matplotlib inline
plot_state_qsphere(state_vec)

In [ ]:
x = np.zeros((8,))
y = np.zeros((8,))

r = x*y
r.shape

In [ ]:
from scripts.util import create_gate
a_con = create_gate(a_norm, n_qubits)
b_con = create_gate(b_norm, n_qubits)

In [ ]:
# psi circuit
ampl_psi = np.concatenate((a_norm/np.linalg.norm(a_norm), b_norm/np.linalg.norm(b_norm)))*(1/np.sqrt(2)) # inputs are normalized
ampl_psi

In [ ]:
# psi circuit
#ampl_a = (a_norm/np.linalg.norm(a_norm))
#ampl_b = (b_norm/np.linalg.norm(b_norm))
n_qubits_psi = int(math.ceil(np.log2(len(ampl_psi))))
print(n_qubits_psi)
psi_anc = QuantumRegister(1, name="psi_anc")
psi_state = QuantumRegister(n_qubits_psi, name="psi_state")
qc_psi = QuantumCircuit(psi_anc, psi_state)
qc_psi.h(psi_anc)
qc_psi.append(ampl_psi, psi_state)


In [ ]:
# defining psi
psi_con = QuantumRegister(1, name="psi_con")
psi_state = QuantumRegister(n_qubits, name="psi_state")
psi = QuantumCircuit(psi_con, psi_state, name="psi")
psi.h(psi_con)
psi.append(a_con, psi.qubits)
psi.x(psi_con)
psi.append(b_con, psi.qubits)

In [ ]:
# defining phi
phi = QuantumCircuit(n_qubits + 1, name="phi")
phi.h(0)

In [ ]:
state_vec = Statevector.from_instruction(qc_psi).data
print(f'State Vector of: {state_vec}')
print(f'State Vector of: {Statevector.from_instruction(qc_psi).to_dict()}')
vector2latex(state_vec, pretext="|%s\\rangle ="%'PSI')

In [ ]:
# phi circuit
ampl_phi = np.array([np.linalg.norm(a_norm), -np.linalg.norm(b_norm)])/np.sqrt(Z)
n_qubits_phi = 1
qc_phi = QuantumCircuit(n_qubits+ n_qubits_phi) # always 1
qc_phi.initialize(ampl_phi, [0])
#state_phi = print_state_vector(qc_phi, 'phi')

In [ ]:
state_vec = Statevector.from_instruction(qc_phi).data
print(f'State Vector of: {state_vec}')
print(f'State Vector of: {Statevector.from_instruction(qc_phi).to_dict()}')
vector2latex(state_vec, pretext="|%s\\rangle ="%'PSI')

In [ ]:
anc = QuantumRegister(1, "ancilla")
qr_psi = QuantumRegister(n_qubits+1, "psi")
qr_phi = QuantumRegister(n_qubits+1, "phi") # size always 1
#cr = ClassicalRegister(1, "cr")

# Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = QuantumCircuit(anc, qr_psi, qr_phi, name="k_means")

qc.append(psi, qr_psi)
qc.append(phi, qr_phi)
print(qc.reverse_bits().draw())
state_vec = Statevector.from_instruction(qc.reverse_bits()).data

In [ ]:
state_vec.shape

In [ ]:
vector2latex(state_vec, pretext="|%s\\rangle ="%'AncPsiPhi')

In [ ]:
qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

In [ ]:
#QIBO
from qibo import *
from qibo.models import Circuit
from qibo import gates

In [ ]:
list_psi_qubits = list(range(1,n_qubits+1+1))
list_phi_qubits = list(range(n_qubits+1+1, n_qubits+1+n_qubits+1+1))

In [ ]:
list_psi_qubits, list_phi_qubits

In [ ]:
#psi circuit
qc_psi = Circuit(n_qubits+1)
    
# phi circuit
qc_phi = Circuit(n_qubits+1)

# create QKmeans circuit
qc = Circuit(2*n_qubits+2+1) # 1 = ancilla psi, 1 = ancilla

qc.add(qc_psi.on_qubits(*(list_psi_qubits)))
qc.add(qc_phi.on_qubits(*(list_phi_qubits)))

In [ ]:
qc.add(gates.H(0))

In [ ]:
for i, q in enumerate(list_psi_qubits):
    print(q)
    print(list_phi_qubits[i])
    qc.add(gates.SWAP(
        q,
        list_phi_qubits[i]
    ).controlled_by(0))

In [ ]:
qc.add(gates.H(0))
    
qc.add(gates.M(0)) # returing back one number!

In [ ]:
result = qc.execute(initial_state=state_vec, nshots=1000)
counts = result.frequencies(binary=True)

In [ ]:
counts

In [ ]:
Z=2.0
distance = 2*(counts['0']/1000 - 0.5)*2*Z

In [ ]:
distance = distc.DistCalc(a, b)

In [ ]:
distance

In [ ]:
# Euclidian distance
dist = np.linalg.norm(a_norm-b_norm)
dist

### Plot DistCalc cirucit

In [ ]:
Z = calc_Z(a_norm, b_norm)
    
psi = create_psi(a_norm, b_norm)
phi = create_phi(a_norm, b_norm)

anc = QuantumRegister(1, "ancilla")
qr_psi = QuantumRegister(psi.width(), "psi")
qr_phi = QuantumRegister(1, "phi") # size always 1
#cr = ClassicalRegister(1, "cr")

# Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = QuantumCircuit(anc, qr_psi, qr_phi, name="k_means")

qc.append(psi, qr_psi)
qc.append(phi, qr_phi)
state_vec = Statevector.from_instruction(qc.reverse_bits()).data
return np.asarray(state_vec), psi.width()

### 2. Check Durr&Hoyer algo

In [ ]:
a = np.array([1,2], dtype=np.float64)
b = np.array([2,3], dtype=np.float64)
num_features = 2

In [ ]:
a = qkm.prepare_input(a)
b = qkm.prepare_input(b)

In [ ]:
n_qubits_psi = len(np.concatenate((a,b)))

In [ ]:
amplitudes = qkm.get_amplitudes_from_qiskit(a, b, num_features)

In [ ]:
amplitudes

In [ ]:
# state_vector_check = qkm.DistCalc_test(a, b, num_features, ampls)
# print(state_vector_check)

In [ ]:
distance, qc = qkm.DistCalc(a, b, num_features)

In [ ]:
print(distance)

In [ ]:
# Euclidian distance
dist = np.linalg.norm(a-b)
dist

In [ ]:
dist = np.sqrt(np.sum(np.square(a - b)))
dist

In [ ]:
# 10 features
distances = np.array([13.2,3.6,4.5,1,8,53,2,19], dtype=np.float32)

In [ ]:
index = m.duerr_hoyer_algo(distances)

In [ ]:
index

### 3. Check QKmeans algo

In [ ]:
samples_n = int(250)
data_bg = np.random.multivariate_normal(mean=(1,1), cov=np.eye(2)*0.2, size=samples_n)
data_sig = np.random.multivariate_normal(mean=(2,2), cov=np.eye(2)*0.5, size=samples_n)

In [ ]:
import importlib
importlib.reload(o)

In [ ]:
p.plot_clusters(np.vstack([data_bg, data_sig]), np.vstack([np.zeros((samples_n,1)), np.ones((samples_n,1))]))

In [ ]:
data = np.concatenate((data_bg, data_sig))
np.random.shuffle(data)

In [ ]:
data.shape

In [ ]:
centroids = qkmed.initialize_centroids(data, k=2)   # Intialize centroids

In [ ]:
centroids = centroids.astype('float64')
data = data.astype('float64')

In [ ]:
plt.figure()
plt.scatter(data[:, 0], data[:, 1], s=40)
plt.scatter(centroids[0,0], centroids[0,1], s=40, c='y', marker='o')
plt.scatter(centroids[1,0], centroids[1,1], s=40, c='r', marker='o')
plt.show()

In [ ]:
# run k-means algorithm
i = 0
while True:
    cluster_label, _ = qkmed.find_nearest_neighbour(data[:2],centroids)       # find nearest centers
    new_centroids = qkmed.find_centroids_GM(data[:2],cluster_label, clusters=2)               # find centroids
    #print('[clustering_quantum:train_qmeans] >>> iter {}: new centers {}'.format(i,new_centroids))
    if np.allclose(new_centroids, centroids, rtol=1E-2):
        break
    print(f'Iteration: {i}')    
    centroids = new_centroids
    i += 1
    #qkm.draw_plot_w_centroids(data, centroids, cluster_label)

In [ ]:
new_centroids = qkmed.find_centroids(data,cluster_label, clusters=2)

In [ ]:
y = np.mean(data[:10], 0)
y.shape

In [ ]:
[y]

In [ ]:
from scipy.spatial.distance import cdist, euclidean

D = cdist(data[:10], [y])
D.shape

In [ ]:
def find_distance_matrix_quantum(XA, XB):
    XA = np.asarray(XA)
    XB = np.asarray(XB)

    sA = XA.shape
    sB = XB.shape

    if len(sA) != 2:
        raise ValueError('XA must be a 2-dimensional array.')
    if len(sB) != 2:
        raise ValueError('XB must be a 2-dimensional array.')
    if sA[1] != sB[1]:
        raise ValueError('XA and XB must have the same number of columns '
                         '(i.e. feature dimension.)')

    mA = sA[0]; mB = sB[0]; n = sA[1]
    
    dist_matrix = np.zeros((mA, mB))
    for i in range(mA):
        dist_matrix[i,:], _ = distc.DistCalc(XA[i,:], XB[0,:], n)
    return dist_matrix
    

In [ ]:
X=data[:10]

In [ ]:
y = np.mean(X, 0)

while True:
    D = find_distance_matrix_quantum(X, [y])
    print(D)
    nonzeros = (D != 0)[:, 0]

    Dinv = 1 / D[nonzeros]
    Dinvs = np.sum(Dinv)
    W = Dinv / Dinvs
    T = np.sum(W * X[nonzeros], 0)

    num_zeros = len(X) - np.sum(nonzeros)
    if num_zeros == 0:
        y1 = T
    elif num_zeros == len(X):
        end = y
        break
    else:
        R = (T - y) * Dinvs
        r = np.linalg.norm(R)
        rinv = 0 if r == 0 else num_zeros/r
        y1 = max(0, 1-rinv)*T + min(1, rinv)*y
    
    d_y_y1,_ = distc.DistCalc(y, y1, y.shape[0])
    if d_y_y1 < 1e-2:
        end = y1
        break

    y = y1

In [ ]:
end

In [ ]:
y

In [ ]:
cluster_label

In [ ]:
qkm.draw_plot_w_centroids(data, centroids, cluster_label) #only 2 feature data

In [ ]:
p.plot_latent_representations(data, cluster_label, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'test_clusters_qkmedians_DURRmin' )

In [ ]:
# Check classical Kmeans
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=2).fit(data)
    
centroids = kmean.cluster_centers_
pred_kmean = kmean.predict(data)

p.plot_latent_representations(data, pred_kmean, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'test_clusters_kmeans' )

### 4. Check QKmeans on Dijet dataset

In [ ]:
import h5py
from sklearn.model_selection import train_test_split

In [ ]:
save_dir='baseline_results/models/AE/test_26012022'
output_file = f'/eos/user/e/epuljak/private/epuljak/PhD/Kinga/{save_dir}/latentsamples_QCD_signal.h5'

In [ ]:
with h5py.File(output_file,'r') as file:
    #file['predicted_QCD'][:]
    latent_space = file['encoded_QCD'][:]

In [ ]:
latent_space.shape

In [ ]:
num_samples = 100000

In [ ]:
X_train, X_test = train_test_split(latent_space[:num_samples], test_size=0.2, shuffle=False)

In [ ]:
k = 2 # number of clusters
centroids = qkm.initialize_centroids(X_train, k=k)   # Intialize centroids

In [ ]:
centroids = centroids.astype('float64')
X_train = X_train.astype('float64')

In [ ]:
X_train.shape

In [ ]:
# run qk-means algorithm
i = 0
while True:
    cluster_label, _ = qkm.find_nearest_neighbour(X_train,centroids)       # find nearest centers
    new_centroids = qkm.find_centroids(X_train,cluster_label, clusters=2)               # find centroids
    if np.allclose(new_centroids, centroids, rtol=1.e-2):
            break
            
    centroids = new_centroids
    i += 1
    break
    #qkm.draw_plot_w_centroids(X_train, centroids, cluster_label)
print('Finished')
#qkm.draw_plot_w_centroids(X_train, centroids, cluster_label)

In [ ]:
new_centroids = qkm.find_centroids(X_train,cluster_label, clusters=2)

In [ ]:
p.plot_latent_representations(X_train, cluster_label, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'jets_test_qkmeans' )

In [ ]:
cluster_label.shape

In [ ]:
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=2, n_init=1, max_iter=1).fit(X_train)
    
centroids = kmean.cluster_centers_
pred_kmean = kmean.predict(X_train)

p.plot_latent_representations(X_train, pred_kmean, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'jets_test_kmeans' )

## 5. Check KMeans on Dijet dataset

In [ ]:
save_dir='inference_ntb/results/01022022_1'
output_file = f'/eos/user/e/epuljak/private/epuljak/PhD/Autoencoders/{save_dir}/latentrep_QCD_sig.h5'

In [ ]:
with h5py.File(output_file,'r') as file:
    latent_space = file['latent_space'][:]

In [ ]:
latent_space.shape

In [ ]:
# Check classical Kmeans
from sklearn.cluster import KMeans

kmean = KMeans(n_clusters=2).fit(latent_space[:10000])
    
centroids = kmean.cluster_centers_
pred_kmean = kmean.predict(latent_space[10000:20000])

p.plot_latent_representations(latent_space[10000:20000], pred_kmean, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'test_clusters_kmeans_DAE' )

In [ ]:
import importlib
importlib.reload(KM)

In [ ]:
# Check implementation from GITHUB
import scripts.kmeans as KM

kmeans_gh = KM.Kmeans(k=2)
kmeans_gh.fit(latent_space[:50000])
    
#centroids = kmean_gh.cluster_centers_
pred_kmean = kmeans_gh.predict(latent_space[50000:100000])

p.plot_latent_representations(latent_space[50000:100000], pred_kmean, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'test_clusters_kmeansGH_DAE' )

## 6. Check KMedians and QKMedians for Dijet data

In [ ]:
run='01022022_1'

In [ ]:
save_dir='inference_ntb/results/01022022_1'
output_file = f'/eos/user/e/epuljak/private/epuljak/PhD/Autoencoders/{save_dir}/latentrep_QCD_sig.h5'

In [ ]:
with h5py.File(output_file,'r') as file:
    latent_space = file['latent_space'][:]

In [ ]:
num_samples = 100

In [ ]:
data = latent_space[:num_samples]

In [ ]:
# Check implementation from GITHUB
import scripts.kmedians as KMed

kmedians = KMed.Kmedians(k=2)
kmedians.fit(data)
    
#centroids = kmean_gh.cluster_centers_
pred_kmedians = kmedians.predict(data)

p.plot_latent_representations(data, pred_kmedians, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', f'jets_kmedians_{run}_10000' )

In [ ]:
k = 2 # number of clusters
tolerance = 1e-2
centroids = qkmed.initialize_centroids(latent_space[:num_samples], k=k)   # Intialize centroids

In [ ]:
centroids = centroids.astype('float64')
latent_space = latent_space.astype('float64')

In [ ]:
import importlib
importlib.reload(qkmed)

In [ ]:
# run qk-medians algorithm
i = 0
while True:
    cluster_label, _ = qkmed.find_nearest_neighbour(latent_space[:num_samples],centroids)       # find nearest centers
    new_centroids = qkmed.find_centroids_GM(latent_space[:num_samples],cluster_label, clusters=2)               # find centroids
    
    if np.linalg.norm(centroids - new_centroids) < tolerance:
        centroids = new_centroids
        print(f"Converged after {i} iterations.")
        break
    i+=1       
    centroids = new_centroids
    
print('Finished')


In [ ]:
cluster_label_test, _ = qkm.find_nearest_neighbour(latent_space[num_samples:num_samples+1000],centroids)       # find nearest centers

In [ ]:
p.plot_latent_representations(latent_space[:num_samples], cluster_label, '/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/plots', 'test_clusters_qkmedians' )

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

## Check geometric median function

In [ ]:
distances = np.array([[1,2,3,4,5,6,7,8],[2,4,6,8,10,12,14,16],[1,3,5,7,9,11,13,15]], dtype=np.float32)

In [ ]:
median_np = np.median(distances,axis=0)
median_np

In [ ]:
def check_similarity(y, X, tol=1e-2):
    for i in range(X.shape[0]):
        if np.isclose(y, X[i,:], rtol=tol).all():
            continue
        else:
            print("Not close!")
            return False
    return True

In [ ]:
def geometric_median(X, eps=1e-6):
    if X.size==0: 
        print("For this class there is no points assigned!")
        return
    y = np.mean(X, 0)
    print(f'First median is: {y}')
    z=0
    while True:
        #print(z)
        D = qkmed.find_distance_matrix_quantum(X, [y])
        #print(D)
        nonzeros = (D != 0)[:, 0] #which points are not equal to y
        Dinv = 1 / D[nonzeros]
        Dinv_sum = np.sum(Dinv)
        W = Dinv / Dinv_sum
        T1 = np.sum(W * X[nonzeros], 0) #scaled sum of all points
        
        num_zeros = len(X) - np.sum(nonzeros) # number of points = y
        #print(f'Size of inputs: {X.shape}, and there is 0: {num_zeros}')
        if num_zeros == 0: #then next median is scaled sum of all points
            y1 = T1
        elif num_zeros == len(X): # if all points are same as median
            return y
        else:
            R = (T1 - y) * Dinv_sum
            r = np.linalg.norm(R)
            gamma = 0 if r == 0 else num_zeros/r
            gamma = min(1, gamma)
            y1 = (1-gamma)*T1 + gamma*y
        
        # converge condition    
        dist_y_y1,_ = distc.DistCalc_AmplE(y, y1)
        print(f'Distance between 2 medians: {dist_y_y1}')
        if dist_y_y1 < eps:
            if check_similarity(y1, X):
                return y1
        #print(f'Next median is: {y1}')
        y = y1 # next median is
        z+=1

In [ ]:
median = geometric_median(distances)

In [ ]:
median